# Stage 1 - Data Cleaning and Preprocessing

### Team 7 - Nicole Bolliger, Vitalia Vedenikova



 ## 1. Enhanced Data Cleaning, Preprocessing, and Exploratory Analysis

 ---

### 1.1. Requirements and Data Collection

---



In [ ]:
#Installations
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install nltk
!pip install gensim
!pip install langdetect
!pip install beautifulsoup4
!pip install num2words
!pip install contractions
!pip install unidecode
!pip install pattern3
!pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached pyspellchecker-0.8.1-py3-none-any.whl.metadata (9.4 kB)
Using cached pyspellchecker-0.8.1-py3-none-any.whl (6.8 MB)


In [ ]:
# Import required packages
import pandas as pd
import numpy as np
import zipfile
import json
import nltk
import unidecode
import spacy

from spellchecker import SpellChecker

from gensim.utils import tokenize
from gensim.parsing.preprocessing import remove_stopwords

from nltk.corpus import wordnet as wn
from nltk import pos_tag
from collections import defaultdict
from nltk.stem import WordNetLemmatizer

try:
    nltk.data.find('taggers/averaged_perceptron_tagger_eng/')
except LookupError:
    nltk.download('averaged_perceptron_tagger_eng')

nltk.download('wordnet')
nltk.download('punkt_tab')

from langdetect import detect, detect_langs, DetectorFactory
from collections import Counter
from collections import defaultdict

import spacy
import re
from bs4 import BeautifulSoup
import string
from num2words import num2words
import contractions


pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip -q $zip_path -d /content/project_dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Example: Adjust the path based on where you uploaded the file in Google Drive
zip_path = '/content/drive/My\ Drive/CLTC_Project/content/project_dataset.zip'

# Unzip the file
!unzip -q $zip_path -d /content/project_dataset


replace /content/project_dataset/project_dataset/cleantech_media_dataset_v3_2024-10-28.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/project_dataset/__MACOSX/project_dataset/._cleantech_media_dataset_v3_2024-10-28.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/project_dataset/project_dataset/cleantech_rag_evaluation_data_2024-09-20.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/project_dataset/__MACOSX/project_dataset/._cleantech_rag_evaluation_data_2024-09-20.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [ ]:
!ls /content/project_dataset/project_dataset


cleantech_media_dataset_v3_2024-10-28.csv  cleantech_rag_evaluation_data_2024-09-20.csv


In [ ]:
#Import csv data into dataframes
df_media = pd.read_csv('/content/project_dataset/project_dataset/cleantech_media_dataset_v3_2024-10-28.csv')
#df_media_rag = pd.read_csv('/content/project_dataset/project_dataset/cleantech_rag_evaluation_data_2024-09-20.csv', on_bad_lines='skip')

In [ ]:
# Import JSON data into a dataframe

import json

patent_json = []  # Initialize patent_json outside the try-except block

file_path = '/content/drive/My Drive/CLTC_Project/content/CleanTech_22-24_updated.json'

with open(file_path) as f:
    try:
        # Try loading the entire file as a single JSON object
        patent_json = json.load(f)
    except json.JSONDecodeError:
        # If loading as a single JSON object fails
        f.seek(0)  # Reset the file pointer to the beginning

        # Iterate through each line in the file
        for line in f:
            try:
                # Attempt to parse each line as a JSON object
                data = json.loads(line)
                # Append the parsed JSON object to patent_json list
                patent_json.append(data)
            except json.JSONDecodeError as e:
                # If a line is not valid JSON, print an error message
                print(f"Skipping invalid line: {line.strip()} - Error: {e}")

# Convert dictionary to JSON string
json_string = json.dumps(patent_json)

# Convert nested JSON data to Pandas DataFrame
df_patent = pd.read_json(json_string, orient='records')

Skipping invalid line: ion_date":"20230802","inventor":["RIVA, Matteo"],"cpc_code":"H01L31/0488"} - Error: Expecting value: line 1 column 1 (char 0)
Skipping invalid line: {"publication_number":"CN-108396729-B","application_number":"CN-201810424026-A","country_code":"CN","title":"Upward deep drainage hole protection device and installation construction method","abstract":"The invention relates to t - Error: Unterminated string starting at: line 1 column 203 (char 202)


<ipython-input-4-73ac687e0b8f>:34: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_patent = pd.read_json(json_string, orient='records')


### 1.2. Data Cleaning

---

#### 1.2.1. Googe Patents Dataset

In [ ]:
# Examine google patent dataframe
df_patent.head(1)

,publication_number,application_number,country_code,title,abstract,publication_date,inventor,cpc_code
0,CN-117138249-A,CN-202311356270-A,CN,一种石墨烯光疗面罩,"The application provides a graphene phototherapy mask, and relates to the technical field of phototherapy devices of graphene phototherapy masks. The solar energy heat conducting mask comprises a mask outer shell, a lamp panel, a graphene heating plate and a transparent heat conducting mask inner shell which are sequentially arranged from outside to inside; a plurality of lamp beads are arranged on the lamp panel; a plurality of through holes are formed in the graphene heating plate at positions corresponding to the lamp beads; a controller provided with a control board therein; the control panel is provided with a PWM micro-pulse control circuit and a heating control circuit, the PWM micro-pulse control circuit is electrically connected to the lamp panel and used for driving the lamp beads to emit micro-pulse light, the heating control circuit is electrically connected to the graphene heating plate and used for driving the graphene heating plate to heat, and the pulse frequency of the micro-pulse light is greater than or equal to 50Hz. The graphene phototherapy mask is driven by the heating function and the micropulse light, so that the phototherapy effect can be improved, the skin problem is improved, the use experience is improved, and better phototherapy effect and comfort are brought to users.",20231201,"[LI HAITAO, CAO WENQIANG]",A61N2005/0654


The dataset includes the following fields for each patent record:

- publication_number: The publication number of the patent. Publication numbers are generally made up of a country code (two letters) and a serial number (variable, 1-12 digits).

- application_number: The application number corresponding to the patent.

- country_code: The code representing the country in which the patent was filed.

- title_localized: The title of the patent in its localized form.

- abstract_localized: A localized abstract describing the patent.

- publication_date: The date on which the patent was published.

- inventor: The inventor(s) of the patent.

- cpc: Cooperative Patent Classification code(s) associated with the patent. Patent publications are each assigned at least one classification term indicating the subject to which the invention relates. They may also be assigned additional classification terms to give further details on the contents.


Since cpc codes assign subjects to the patents, they can potentially serve as the ground truth and help us evaluate any models we may use.

##### Remove Duplicate Rows

The amount of duplicates based on the publication number is 375'491.

In [ ]:
df_patent.duplicated(subset=['publication_number']).sum()

375491

By examining some of the duplicates, we realise that the "cpc_code" field changes. Therefore, we eliminate duplicates by keeping only one row per "publication_number" but we concatenate the cpc codes to be able to analyse them at a later stage.

In [ ]:
df_patent[df_patent['publication_number']=='CN-117138249-A']

,publication_number,application_number,country_code,title,abstract,publication_date,inventor,cpc_code
0,CN-117138249-A,CN-202311356270-A,CN,一种石墨烯光疗面罩,"The application provides a graphene phototherapy mask, and relates to the technical field of phototherapy devices of graphene phototherapy masks. The solar energy heat conducting mask comprises a mask outer shell, a lamp panel, a graphene heating plate and a transparent heat conducting mask inner shell which are sequentially arranged from outside to inside; a plurality of lamp beads are arranged on the lamp panel; a plurality of through holes are formed in the graphene heating plate at positions corresponding to the lamp beads; a controller provided with a control board therein; the control panel is provided with a PWM micro-pulse control circuit and a heating control circuit, the PWM micro-pulse control circuit is electrically connected to the lamp panel and used for driving the lamp beads to emit micro-pulse light, the heating control circuit is electrically connected to the graphene heating plate and used for driving the graphene heating plate to heat, and the pulse frequency of the micro-pulse light is greater than or equal to 50Hz. The graphene phototherapy mask is driven by the heating function and the micropulse light, so that the phototherapy effect can be improved, the skin problem is improved, the use experience is improved, and better phototherapy effect and comfort are brought to users.",20231201,"[LI HAITAO, CAO WENQIANG]",A61N2005/0654
58,CN-117138249-A,CN-202311356270-A,CN,Graphene phototherapy mask,"The application provides a graphene phototherapy mask, and relates to the technical field of phototherapy devices of graphene phototherapy masks. The solar energy heat conducting mask comprises a mask outer shell, a lamp panel, a graphene heating plate and a transparent heat conducting mask inner shell which are sequentially arranged from outside to inside; a plurality of lamp beads are arranged on the lamp panel; a plurality of through holes are formed in the graphene heating plate at positions corresponding to the lamp beads; a controller provided with a control board therein; the control panel is provided with a PWM micro-pulse control circuit and a heating control circuit, the PWM micro-pulse control circuit is electrically connected to the lamp panel and used for driving the lamp beads to emit micro-pulse light, the heating control circuit is electrically connected to the graphene heating plate and used for driving the graphene heating plate to heat, and the pulse frequency of the micro-pulse light is greater than or equal to 50Hz. The graphene phototherapy mask is driven by the heating function and the micropulse light, so that the phototherapy effect can be improved, the skin problem is improved, the use experience is improved, and better phototherapy effect and comfort are brought to users.",20231201,"[LI HAITAO, CAO WENQIANG]",A61N2005/0626
323,CN-117138249-A,CN-202311356270-A,CN,一种石墨烯光疗面罩,"The application provides a graphene phototherapy mask, and relates to the technical field of phototherapy devices of graphene phototherapy masks. The solar energy heat conducting mask comprises a mask outer shell, a lamp panel, a graphene heating plate and a transparent heat conducting mask inner shell which are sequentially arranged from outside to inside; a plurality of lamp beads are arranged on the lamp panel; a plurality of through holes are formed in the graphene heating plate at positions corresponding to the lamp beads; a controller provided with a control board therein; the control panel is provided with a PWM micro-pulse control circuit and a heating control circuit, the PWM micro-pulse control circuit is electrically connected to the lamp panel and used for driving the lamp beads to emit micro-pulse light, the heating control circuit is electrically connected to the graphene heating plate and used for driving the graphene heating plate to heat, and the pulse fre

In [ ]:
df_patent = df_patent.groupby('publication_number', as_index=False).agg({
    'application_number': 'first',  # Keep the first application number
    'country_code': 'first',        # Keep the first country code
    'title': 'first',     # Keep the first title
    'abstract': 'first',  # Keep the first abstract
    'publication_date': 'first',    # Keep the first publication date
    'inventor': 'first',           # Keep the first inventor
    'cpc_code': lambda x: ', '.join(x.astype(str))  # Concatenate 'cpc' values
})

In [ ]:
len(df_patent)

31366

##### Remove application_number

In [ ]:
df_patent = df_patent.drop('application_number', axis= 1)

##### Correct Data Types

In [ ]:
#The publication date is an integer
df_patent['publication_date'].dtype

dtype('int64')

In [ ]:
# correct it to a date
df_patent['publication_date'] = pd.to_datetime(df_patent['publication_date'], format='%Y%m%d')

In [ ]:
#The title, abstract and inventor columns are object data types
df_patent[['abstract', 'title', 'inventor']].dtypes

,0
abstract,object
title,object
inventor,object


In [ ]:
# We cast them as string data types
columns_to_cast = ['abstract', 'title', 'inventor']

for column in columns_to_cast:
    df_patent[column] = df_patent[column].astype('string')

df_patent[['abstract', 'title','inventor']].dtypes

,0
abstract,string[python]
title,string[python]
inventor,string[python]


##### Keep Only Rows Where Abstracts Are In English

In [ ]:
len(df_patent)

31366

In [ ]:
# We check if all the abstracts are in english
detected_languages = [detect_langs(text) for text in df_patent['abstract'].astype(str)]
df_patent['detected_language'] = detected_languages

DetectorFactory.seed = 0

flattened_languages = [lang.lang[:2] if lang else 'unknown' for sublist in detected_languages for lang in sublist]

counts = Counter(flattened_languages)

language_stats = pd.DataFrame.from_dict(counts, orient='index', columns=['Count'])
en_count = language_stats.loc[language_stats.index.str.startswith('en', na=False), 'Count'].sum()
print(f"Number of times 'en' appears: {en_count}")

Number of times 'en' appears: 28880


In [ ]:
# All languages that are not "en":
filtered_df = df_patent[~df_patent['detected_language'].astype(str).str.contains('en')]

#All unique language codes
filtered_df['language_code'] = filtered_df['detected_language'].apply(lambda langs: [lang.lang[:2] for lang in langs if lang])
unique_language_codes = filtered_df['language_code'][:2].explode().unique()
print(unique_language_codes)

['fr']


<ipython-input-37-bed1a83dbee0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['language_code'] = filtered_df['detected_language'].apply(lambda langs: [lang.lang[:2] for lang in langs if lang])


In [ ]:
# We keep only the rows where the abstracts are in English (28'881 out of 31'366 records)
df_patent['is_en'] = df_patent['detected_language'].apply(lambda langs: any(lang.lang.startswith('en') for lang in langs))
df_patent = df_patent[df_patent['is_en']]
len(df_patent)

28880

##### Clean Inventor Column

In [ ]:
def clean_inventor_names(name_string):
  cleaned_string = re.sub(r"[\[\]']", "", name_string)
  return cleaned_string

df_patent['inventor'] = df_patent['inventor'].apply(clean_inventor_names)

#### 1.2.2. Media Dataset

In [ ]:
# Examine media dataframe
df_media.head(3)

Unnamed: 0  \
0       93320   
1       93321   
2       98159   

                                                                                 title  \
0                                            XPeng Delivered ~100,000 Vehicles In 2021   
1                                        Green Hydrogen: Drop In Bucket Or Big Splash?   
2  World’ s largest floating PV plant goes online in China – pv magazine International   

         date  author  \
0  2022-01-02     NaN   
1  2022-01-02     NaN   
2  2022-01-03     NaN   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

##### Remove author and unnamed columns

In [ ]:
# The author column is empty
unique_authors = df_media['author'].unique()
unique_authors

array([nan])

In [ ]:
# remove author and unnamed column
df_media = df_media.drop(['author','Unnamed: 0'], axis= 1)

##### Check for duplicates

In [ ]:
num_duplicates = df_media.duplicated().sum()
print(f"Number of duplicates: {num_duplicates}")

Number of duplicates: 0


##### Correct Data Types

In [ ]:
#The title and content are both object data types
df_media[['content', 'title']].dtypes

,0
content,object
title,object


In [ ]:
# We cast them as string data types
columns_to_cast = ['content', 'title']

for column in columns_to_cast:
    df_media[column] = df_media[column].astype('string')

df_media[['content', 'title']].dtypes

,0
content,string[python]
title,string[python]


### 1.3. Text Preprocessing

---

#### 1.3.1. Patent Data

##### Lowercasing

In [ ]:
# Lowercasing
df_patent['abstract'] = df_patent['abstract'].str.lower()
df_patent['title'] = df_patent['title'].str.lower()

###### Remove Punctuation

In [ ]:
PUNCTUATIONS = string.punctuation

def remove_punctuation(df, section):
  df[f'{section}'] = df[f'{section}'].apply(lambda x: x.translate(str.maketrans('', '', PUNCTUATIONS)))
  return df

df_patent = remove_punctuation(df_patent, 'abstract')
df_patent = remove_punctuation(df_patent, 'title')

##### Replace Symbols (e.g. '%', '~', '$') and Abbreviations by Words

In [ ]:
symbol_mapping = {
       '%': ' percent',
       'gw': ' gigawatt',
       'mw': ' megawatt',
       'kw': ' kilowatt',
       'm2': ' square meter',
       'kg': ' kilogram',
       'm3': ' cubic meter',
       '°f': ' degree fahrenheit',
       '°c': ' degree celsius',
       '°k': ' degree kelvin',
       'gwdc': ' gigawatts-direct current',
       'mwp' : ' megawatt peak',
       'mwh' : 'megawatt-hour',
       'gwh': ' gigawatt-hour',
       'hz': ' hertz'
   }

def replace_symbols(text, symbol_map):
  for symbol, replacement in symbol_map.items():
    text = text.replace(symbol, replacement)
  return text

df_patent['abstract'] = df_patent['abstract'].apply(lambda x: replace_symbols(x, symbol_mapping))
df_patent['title'] = df_patent['title'].apply(lambda x: replace_symbols(x, symbol_mapping))

###### Remove Extra Whitespaces

In [ ]:
def remove_whitespaces(text):
  return " ".join(text.split())
df_patent['abstract'] = df_patent['abstract'].apply(remove_whitespaces)
df_patent['title'] = df_patent['title'].apply(remove_whitespaces)

###### Remove Stopwords

In [ ]:
# Remove stopwords using gensim
df_patent['abstract'] = df_patent['abstract'].apply(lambda x: remove_stopwords(x))

# Print the lengths of the filtered columns and the first few rows to check the result
print(len(df_patent['abstract']), df_patent['abstract'].head())

28880 0    closed loop control heliostats abstract described control solar energy collection apparatus 1 apparatus includes plurality individually angularly controllable heliostats 3 disposed 5 adjacent central receiver 5 heliostats able tilted reflect solar energy target region central receiver 5 beam solid angle control includes measuring device 23 capable scanned surface central receiver 5 including target region measuring device 23 includes distribution 31 photo sensors 33 configured sense presence beam 10 heliostat control includes controller 21 performing closedloop control procedure 43 iteratively vary pointing direction heliostat bring respective beam target region 26 33 25 27 5 9 23 3 1 29 35 7 15 19 13 21 17 figure 2
1                                                                                                                    invention relates device capturing solar energy including plurality singleaxis solar trackers s1s5 having long structure 4 plurality photovoltaic 

###### Replace Numbers by Words

In [ ]:
# Remove superscript characters
df_patent['abstract'] = df_patent['abstract'].apply(lambda x: re.sub(r"[\u00B9\u00B2\u00B3\u2018\u2019\u201A\u201B\u2032\u2033\u2034\u2035]", '', x))

In [ ]:
# Now replace numbers with words
def replace_numbers_with_words (df, column_name):
  for index, row in df.iterrows():
    text = row[column_name]
    new_text = []
    for word in text.split():
        if word.isdigit() and word.isnumeric():
            new_text.append(num2words(word))
        else:
            new_text.append(word)
    df.loc[index, column_name] = " ".join(new_text)
  return df

df_patent = replace_numbers_with_words(df_patent, 'abstract')
df_patent = replace_numbers_with_words(df_patent, 'title')

###### Expand Contractions

In [ ]:
def expand_contractions(text):
  return contractions.fix(text)
df_patent['abstract'] = df_patent['abstract'].apply(expand_contractions)

###### Remove Unnecessary Columns

In [ ]:
df_patent.drop('detected_language', axis=1, inplace=True)
df_patent.drop('is_en', axis=1, inplace=True)

#### 1.3.2. Tokenization

We noticed that we still had some punctuation in the abstracts. We found that tokenizing using gensim removes most punctation (unlike with spacy). Therefore, we use it here:

In [ ]:
#Using gensim for tokenization to avoid picking up any remaining punctuation
df_patent['abstract'] = df_patent['abstract'].apply(lambda x: list(tokenize(x)))

#### 1.3.3. Lemmatization

In [ ]:
# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Lemmatization using SpaCy with punctuation removal
def lemmatize_with_spacy(text):
    if isinstance(text, list):  # Check if text is a list
        text = " ".join(text)
    doc = nlp(text)  # Process the text with spaCy
    lemmas = [token.lemma_ for token in doc if token.pos_ != 'PUNCT']  # Extract lemmas that are not PUNCT
    return " ".join(lemmas) # Join lemmas

df_patent['abstract'] = df_patent['abstract'].apply(lemmatize_with_spacy)

In [ ]:
#there are still standalone "x" and "s" characters in the token text that fall under PUNCT. We remove them:
def remove_xs_from_tokens(text):
  """Removes the characters 'x' and 's'"""
  tokens = text.split()
  cleaned_tokens = [token for token in tokens if token not in ('x', 's')]
  return " ".join(cleaned_tokens)

df_patent['abstract'] = df_patent['abstract'].apply(remove_xs_from_tokens)

In [ ]:
# We check the categorization of the lemmas
for doc in nlp.pipe(df_patent['abstract'].head(), batch_size=500):
  print([(w.text, w.pos_) for w in doc])

[('application', 'NOUN'), ('provide', 'VERB'), ('graphene', 'ADJ'), ('phototherapy', 'NOUN'), ('mak', 'NOUN'), ('relate', 'VERB'), ('technical', 'ADJ'), ('field', 'NOUN'), ('phototherapy', 'NOUN'), ('device', 'NOUN'), ('graphene', 'ADJ'), ('phototherapy', 'NOUN'), ('mak', 'PROPN'), ('olar', 'ADJ'), ('energy', 'NOUN'), ('heat', 'NOUN'), ('conduct', 'NOUN'), ('mak', 'PROPN'), ('comprie', 'PROPN'), ('mak', 'PROPN'), ('outer', 'NOUN'), ('he', 'PRON'), ('will', 'AUX'), ('lamp', 'VERB'), ('panel', 'NOUN'), ('graphene', 'ADJ'), ('heating', 'NOUN'), ('plate', 'NOUN'), ('tranparent', 'ADJ'), ('heat', 'NOUN'), ('conducting', 'VERB'), ('mak', 'NOUN'), ('inner', 'ADJ'), ('he', 'PRON'), ('will', 'AUX'), ('equentially', 'ADV'), ('arrange', 'VERB'), ('outide', 'ADJ'), ('inide', 'NOUN'), ('plurality', 'NOUN'), ('lamp', 'PROPN'), ('bead', 'PROPN'), ('arrange', 'PROPN'), ('lamp', 'PROPN'), ('panel', 'NOUN'), ('plurality', 'NOUN'), ('hole', 'NOUN'), ('form', 'VERB'), ('graphene', 'ADJ'), ('heating', 'NOU

In [ ]:
# There is still some punctuation
all_pos_categories = set()

for doc in nlp.pipe(df_patent['abstract'], batch_size=500):
    for token in doc:
        all_pos_categories.add(token.pos_)

print(all_pos_categories)

{'PROPN', 'VERB', 'ADJ', 'AUX', 'PRON', 'SCONJ', 'PART', 'DET', 'NUM', 'INTJ', 'PUNCT', 'ADV', 'NOUN', 'ADP', 'CCONJ', 'X'}


In [ ]:
df_patent['abstract'].head()

,abstract
0,close loop control heliostat abstract describe control solar energy collection apparatus one apparatus include plurality individually angularly controllable heliostat three dispose five adjacent central receiver five heliostat able tilted reflect solar energy target region central receiver five beam solid angle control include measure device twenty three capable scanned surface central receiver five include target region measure device twenty three include distribution thirty one photo sensor thirty three configure sense presence beam ten heliostat control include controller twenty one perform closedloop control procedure forty three iteratively vary point direction heliostat bring respective beam target region twenty six thirty three twenty five twenty seven five nine twenty three three one twenty nine thirty five seven fifteen nineteen thirteen twenty one seventeen figure two
1,invention relate device capture solar energy include plurality singleaxis solar tracker have long structure four plurality photovoltaic panel two fit tubular rotate shaft one attach low structure four plurality support foot twenty rotatably support tubular rotate shaft one singleaxis solar tracker coaxially align ground motor rotate tubular rotate shafts one accord relative movement sun positive negative conductive cable a b conduct electrical energy generate photovoltaic panel two connected series different singleaxis solar tracker house inside tubular rotate shafts one
2,window shade include mount bracket configure couple structure shade tube bracket configure rotatably couple mount bracket shade couple shade tube bracket have solar panel solar panel electrically couple external shade tube bracket mount bracket solar panel configure receive solar energy transform solar energy electricity provide electricity external
3,method improve accuracy prediction renewable energy solar energy wind energy available electric utility andor refine prediction provide improved integration meteorological forecast coefficient value calculate renewable energy generation model perform regression analysis forecast level renewable energy post utility forecast weather condition measure seasonality explanatory variable accuracy enhance inclusion large number time series variable reflect systematic nature energyweather model use original forecast post operator variable control season
4,provide ocean carbon capture storage ocean ccs method device perform carbon capture storage respect waste gas exhausted marine facility consume fossil energy seashore power plant ship atmosphere basis geoengineering concept large carbon dioxide store ocean natural engineering technique perform scrub waste gas combustion fossil fuel andor perform carbon capture respect air seawater scrubbing carbon dioxide dissolve control indicator include ph value meet environmental regulation seawater inject epipelagic zone bathypelagic zone abyssopelagic zone carbon storage flow ocean current facilitate storage effect


In [ ]:
#save the result into a csv file
df_patent.to_csv('df_patent.csv', index=False)

In [ ]:
# Save the file to Google Drive
df_patent.to_csv('/content/drive/My Drive/CLTC_Project/content/df_patent.csv', index=False)

#### 1.3.2. Media Data

###### Remove all rows where the articles are not in English

In [ ]:
# We remove all content not in english
print(f"The current amount of articles is : {len(df_media)}")
detected_languages = [detect_langs(text) for text in df_media['content'].astype(str)]
df_media['detected_language'] = detected_languages

DetectorFactory.seed = 0

flattened_languages = [lang.lang[:2] if lang else 'unknown' for sublist in detected_languages for lang in sublist]

counts = Counter(flattened_languages)

language_stats = pd.DataFrame.from_dict(counts, orient='index', columns=['Count'])
en_count = language_stats.loc[language_stats.index.str.startswith('en', na=False), 'Count'].sum()
print(f"Number of times 'en' appears: {en_count}")

df_media['is_en'] = df_media['detected_language'].apply(lambda langs: any(lang.lang.startswith('en') for lang in langs))
df_media = df_media[df_media['is_en']]
print(f"The amount of articles now is: {len(df_media)}")

df_media = df_media.drop(['detected_language','is_en'], axis= 1)

The current amount of articles is : 20111
Number of times 'en' appears: 20108
The amount of articles now is: 20108


##### Lowercasing

In [ ]:
# Lowercasing both the content and the title (the latter might be used for EDA)
df_media['content'] = df_media['content'].str.lower()
df_media['title'] = df_media['title'].str.lower()

###### Remove Punctuation

In [ ]:
df_media = remove_punctuation(df_media, 'content')
df_media = remove_punctuation(df_media, 'title')

###### Remove Quotation Marks and Dashes

In [ ]:
def remove_quotes_dashes(text):
  """Removes quotation marks and dashes"""
  text = re.sub(r'[\"\'\u201D\u201C]', '', text)  # Remove quotation marks
  text = re.sub(r'[\u2013\u2014\u2015]', ' ', text)  # Remove dashes
  return text

df_media['content'] = df_media['content'].apply(remove_quotes_dashes)

###### Remove Emails, URLs, HTML Tags

In [ ]:
# Remove emails
df_media['content'] = df_media['content'].astype(str)

#Emails without spaces between username and domain name:
df_media['content'] = df_media['content'].apply(lambda x: re.sub(r'\S+@\S+', '', x))

#Emails with spaces between username and domain name:
df_media['content'] = df_media['content'].apply(lambda x: re.sub(r'[\w\.-]+(?:\s+)?@(?:\s+)?[\w\.-]+(?:\.[\w\.-]+)+', '', x))

In [ ]:
# Remove URLs
df_media['content'] = df_media['content'].apply(lambda x: re.sub(r'https?://\S+|www\.\S+', '', x))

In [ ]:
# Remove HTML tags
def remove_html(df, section):
    df[f'{section}'] = df[f'{section}'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
    return df

df_media = remove_html(df_media, 'content')

###### Remove Emojis

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df_media['content'] = df_media['content'].apply(lambda x: remove_emoji(x))

###### Remove Text After "©" or "copyright ©"


We also notice that any text after "©" or "copyright ©" usually concerns the website where the information was published rather than the content we want to analyze.

In [ ]:
def remove_text_after_copyright(text):
  match = re.search(r"(©|copyright ©).*", text)
  if match:
    return text[:match.start()]
  return text

df_media['content'] = df_media['content'].apply(remove_text_after_copyright)

###### Remove Domain-Specific Information

Some text at the beginning or the end concerns the subscription service of the website or other site-specific information. After examining the content for each domain, we can filter out this irrelevant text.

We also notice that sometimes the last few strings of text concern the author of the article or contains user comments. Unfortunately, it is quite difficult to remove this.

Furthermore, we see that there are some very few articles (around 5), notably from eurosolar, that are in German instead of English.

In [ ]:
df_media["domain"].unique()

array(['cleantechnica', 'pv-magazine', 'naturalgasintel', 'energyvoice',
       'solarpowerworldonline', 'solarindustrymag', 'thinkgeoenergy',
       'energy-xprt', 'azocleantech', 'pv-tech', 'rechargenews',
       'solarpowerportal.co', 'energyintel', 'greenprophet', 'ecofriend',
       'eurosolar', 'greenairnews', 'all-energy', 'iea', 'decarbxpo',
       'biofuels-news', 'solarquarter', 'storagesummit', 'indorenergy',
       'bex-asia'], dtype=object)

In [ ]:
def remove_text_after_certain_string(text, search_string):
  """Removes text after a specific string from a given text."""
  match = re.search(rf"({search_string}).*", text)
  if match:
    return text[:match.start()]
  return text

def remove_text_between_phrases(text, phrase_1, phrase_2):
  """Removes text between two given phrases."""
  phrase_1 = re.escape(phrase_1)
  phrase_2 = re.escape(phrase_2)
  pattern = rf"{phrase_1}.*?{phrase_2}"
  cleaned_text = re.sub(pattern, "", text, flags=re.DOTALL)
  return cleaned_text

# For pv-magazine
search_string = "this content is protected by copyright and may not be reused"
for index, row in df_media[df_media["domain"] == "pv-magazine"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

# For naturalgasintel
phrase_1 = "sign in to get the best natural gas news"
phrase_2 = "ngi all news access"
for index, row in df_media[df_media["domain"] == "naturalgasintel"].iterrows():
    df_media.loc[index, 'content'] = remove_text_between_phrases(row['content'], phrase_1, phrase_2)

# For solarindustrymag
phrase_1 = "solar industry offers"
phrase_2 = "stay current and navigate the solar market"
for index, row in df_media[df_media["domain"] == "solarindustrymag"].iterrows():
    df_media.loc[index, 'content'] = remove_text_between_phrases(row['content'], phrase_1, phrase_2)

# For energy-xprt
phrase_1 = "no comments"
phrase_2 = "first to comment"
for index, row in df_media[df_media["domain"] == "energy-xprt"].iterrows():
    df_media.loc[index, 'content'] = remove_text_between_phrases(row['content'], phrase_1, phrase_2)

search_string = "energy xprt is a global|need help finding the right suppliers"
for index, row in df_media[df_media["domain"] == "energy-xprt"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

# For azocleantech
search_string = "hi i m azthena you can trust"
for index, row in df_media[df_media["domain"] == "azocleantech"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

# For rechargenews
search_string = "recharge is part of dn media group"
for index, row in df_media[df_media["domain"] == "rechargenews"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

#For solarpowerportal.co
search_string = "thank you for subscribing"
for index, row in df_media[df_media["domain"] == "solarpowerportal.co"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

#For greenprophet
phrase_1 = "windowdojorequire"
phrase_2 = "uniquemethods true"
for index, row in df_media[df_media["domain"] == "greenprophet"].iterrows():
    df_media.loc[index, 'content'] = remove_text_between_phrases(row['content'], phrase_1, phrase_2)

#For ecofriend
search_string = "ecofriendcom a dr prem guides"
for index, row in df_media[df_media["domain"] == "ecofriend"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

#For eurosolar
search_string = "europäische vereinigung"
for index, row in df_media[df_media["domain"] == "eurosolar"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

search_string = "independent of political parties"
for index, row in df_media[df_media["domain"] == "eurosolar"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

#For greenairnews
search_string = "sustainability in the air is the first podcast"
for index, row in df_media[df_media["domain"] == "greenairnews"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

#For all-energy
search_string = "your email address will not be published"
for index, row in df_media[df_media["domain"] == "all-energy"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

#For iea
search_string = "get updates on the iea’ s latest news"
for index, row in df_media[df_media["domain"] == "iea"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

phrase_1 = "create a free iea account"
phrase_2 = "to a paid service"
for index, row in df_media[df_media["domain"] == "iea"].iterrows():
    df_media.loc[index, 'content'] = remove_text_between_phrases(row['content'], phrase_1, phrase_2)

#For decarbxpo
search_string = "to use the full function of this web site"
for index, row in df_media[df_media["domain"] == "decarbxpo"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

#For biofuels-news
search_string = "it's the leading magazine covering all"
for index, row in df_media[df_media["domain"] == "biofuels-news"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)

#For solarquarter
search_string = "this site uses akismet"
for index, row in df_media[df_media["domain"] == "solarquarter"].iterrows():
    df_media.loc[index, 'content'] = remove_text_after_certain_string(row['content'], search_string)


##### Replace Symbols (e.g. '%', '~', '$') and Abbreviations by Words


In [ ]:
symbol_mapping = {
       '$': 'dollar ',
       '~': 'approximately',
       '%': ' percent',
       '£': 'pound ',
       '€': 'euro ',
       '₹': 'rupee ',
       '¥': ' yen',
       'gw': ' gigawatt',
       'mw': ' megawatt',
       'kw': ' kilowatt',
       'm2': ' square meter',
       'kg': ' kilogram',
       'm3': ' cubic meter',
       '°f': ' degree fahrenheit',
       '°c': ' degree celsius',
       '°k': ' degree kelvin',
       'year': ' year',
       '...' : ' ',
       'rmb' : 'renminbi ',
       'gwdc': ' gigawatts-direct current',
       'mwp' : ' megawatt peak',
       'mwh' : 'megawatt-hour',
       'gwh': ' gigawatt-hour',
       'degree': ' degree',
       'hz': ' hertz'
   }

df_media['content'] = df_media['content'].apply(lambda x: replace_symbols(x, symbol_mapping))
df_media['title'] = df_media['title'].apply(lambda x: replace_symbols(x, symbol_mapping))

###### Remove Extra Whitespaces

In [ ]:
df_media['content'] = df_media['content'].apply(remove_whitespaces)

###### Remove Stopwords

In [ ]:
# Remove stopwords using gensim (default ones as well as additional stopwords we noticed in the dataset)
custom_stop_words = {"www", "http", "https", "br", "amazon", "href", "wa", "ha", "like", "just"}

def remove_stopwords_with_custom(text):
    text = remove_stopwords(text)  # Remove default stop words
    text = " ".join(word for word in text.split() if word not in custom_stop_words) # Remove custom stop words
    return text

df_media['content'] = df_media['content'].apply(remove_stopwords_with_custom)

# Print the lengths of the filtered columns and the first few rows to check the result
print(len(df_media['content']), df_media['content'].head())

20108 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

##### Replace Numbers by Words

Numbers are useful to give information about the technology in the patent as well as about financing

In [ ]:
# Remove superscript characters
df_media['content'] = df_media['content'].apply(lambda x: re.sub(r"[\u00B9\u00B2\u00B3\u2018\u2019\u201A\u201B\u2032\u2033\u2034\u2035]", '', x))

In [ ]:
# Now replace numbers with words
df_media = replace_numbers_with_words(df_media, 'content')
df_media = replace_numbers_with_words(df_media, 'title')

##### Expanding Contractions

Since a lot of the articles contain quotes from interviews, there are contractions in the text which we must expand.

In [ ]:
df_media['content'] = df_media['content'].apply(expand_contractions)

In [ ]:
df_media_1 = df_media.copy()

###### Convert accented characters to ASCII characters

In [ ]:
def accented_to_ascii(text):
    return unidecode.unidecode(text)
df_media['content'] = df_media['content'].apply(accented_to_ascii)

###### Correct Spelling

Due to the very long run times of the below function (more than 60 minutes), we decided not to apply it to the dataset but leave it here for information's sake

In [ ]:
#Using Spellchecker

"""
!pip install pyspellchecker
from spellchecker import SpellChecker

spell = SpellChecker()

def correct_spelling(text):
    corrected_text = list()
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        next_word = word
        if word in misspelled_words:
          correction = spell.correction(word)
          next_word = correction if correction is not None else word
        corrected_text.append(next_word)

    return " ".join(corrected_text)

df_media['content'] = df_media['content'].apply(lambda x : correct_spelling(x))
"""

KeyboardInterrupt: 

#### 1.3.2. Tokenization

In [ ]:
#Using gensim to avoid picking up any remaining punctuation
df_media['content'] = df_media['content'].apply(lambda x: list(tokenize(x)))

#### 1.3.3. Lemmatization

In [ ]:
# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Lemmatization using SpaCy with punctuation removal
df_media['content'] = df_media['content'].apply(lemmatize_with_spacy)


In [ ]:
#there are still isolated "x" and "s" in the text that fall under PUNCT. We remove them:
df_media['content'] = df_media['content'].apply(remove_xs_from_tokens)

In [ ]:
# There is still some punctuation
all_pos_categories = set()

for doc in nlp.pipe(df_media['content'], batch_size=500):
    for token in doc:
        all_pos_categories.add(token.pos_)

print(all_pos_categories)

{'PART', 'NOUN', 'X', 'INTJ', 'NUM', 'AUX', 'VERB', 'PROPN', 'CCONJ', 'ADV', 'ADJ', 'PRON', 'DET', 'PUNCT', 'SPACE', 'SYM', 'ADP', 'SCONJ'}


In [ ]:
#save the result into a csv file
df_media.to_csv('df_media.csv', index=False)

In [ ]:
# Save the file to Google Drive
df_media.to_csv('/content/drive/My Drive/CLTC_Project/content/df_media.csv', index=False)

## Declaration of Contributions

Data Preprocessing and Cleaning : Vitalia took the lead with some contributions from Nicole

Exploratory Data Analysis : Nicole took the lead with some contributions from Vitalia

Topic Modeling : Nicole contributed more to 1.5.1. and 1.5.2 and Vitalia to 1.5.3. and 1.5.4.